# data.core

> Fill in a module description here


In [ ]:
#| default_exp data.partitioners

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import os
import sys
import random
import torch
import torchvision
import torchvision.transforms as transforms
from fastcore.utils import *

In [ ]:
#| export
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
#| export
class BasePartitioner:
    def __init__(self, cfg):
        self.cfg = cfg

    def assign(self, dataset_content, dataset_label, dataidx_map, X, y, statistic):
        for client in range(self.cfg.num_clients):
            idxs = dataidx_map[client]
            X[client] = dataset_content[idxs]
            y[client] = dataset_label[idxs]
            for i in np.unique(y[client]):
                statistic[client].append((int(i), int(sum(y[client]==i))))

In [ ]:
#| export
@patch
def parition(self: BasePartitioner, dataset_content, dataset_label, least_samples, dataidx_map):
    "The traditional IID partitioning method"
    X = [[] for _ in range(self.cfg.num_clients)]
    y = [[] for _ in range(self.cfg.num_clients)]
    statistic = [[] for _ in range(self.cfg.num_clients)]
    
    idxs = np.array(range(len(dataset_label)))
    idx_for_each_class = []
    for i in range(self.cfg.data.num_classes):
        idx_for_each_class.append(idxs[self.cfg.data.dataset_label == i])

    class_num_per_client = [self.cfg.data.class_per_client for _ in range(self.cfg.num_clients)]
    for i in range(self.cfg.num_classes):
        selected_clients = []
        for client in range(self.cfg.num_clients):
            if class_num_per_client[client] > 0:
                selected_clients.append(client)
        if len(selected_clients) == 0:
            break
        selected_clients = selected_clients[:int(np.ceil((self.cfg.num_clients/self.cfg.data.num_classes)*self.cfg.data.class_per_client))]

        num_all_samples = len(idx_for_each_class[i])
        num_selected_clients = len(selected_clients)
        num_per = num_all_samples / num_selected_clients
        if self.cfg.balance:
            num_samples = [int(num_per) for _ in range(num_selected_clients-1)]
        else:
            num_samples = np.random.randint(max(num_per/10, least_samples/self.cfg.num_classes), num_per, num_selected_clients-1).tolist()
        num_samples.append(num_all_samples-sum(num_samples))

        idx = 0
        for client, num_sample in zip(selected_clients, num_samples):
            if client not in dataidx_map.keys():
                dataidx_map[client] = idx_for_each_class[i][idx:idx+num_sample]
            else:
                dataidx_map[client] = np.append(dataidx_map[client], idx_for_each_class[i][idx:idx+num_sample], axis=0)
            idx += num_sample
            class_num_per_client[client] -= 1

    X, y, statistic = self.assign(dataset_content, dataset_label, dataidx_map, X, y, statistic)

    return X, y, statistic

In [ ]:
class DirPartitioner(BasePartitioner):
    # https://github.com/IBM/probabilistic-federated-neural-matching/blob/master/experiment.py
    def __init__(self, cfg):
        super().__init__(cfg)
    
    def parition(self, dataset_content, dataset_label, least_samples, dataidx_map):
        X = [[] for _ in range(self.cfg.num_clients)]
        y = [[] for _ in range(self.cfg.num_clients)]
        statistic = [[] for _ in range(self.cfg.num_clients)]

        min_size = 0
        K = self.cfg.data.num_classes
        N = len(dataset_label)

        try_cnt = 1
        while min_size < least_samples:
            if try_cnt > 1:
                print(f'Client data size does not meet the minimum requirement {least_samples}. Try allocating again for the {try_cnt}-th time.')

            idx_batch = [[] for _ in range(self.cfg.num_clients)]
            for k in range(K):
                idx_k = np.where(dataset_label == k)[0]
                np.random.shuffle(idx_k)
                proportions = np.random.dirichlet(np.repeat(self.cfg.data.alpha, self.cfg.num_clients))
                proportions = np.array([p*(len(idx_j)<N/self.cfg.num_clients) for p,idx_j in zip(proportions,idx_batch)])
                proportions = proportions/proportions.sum()
                proportions = (np.cumsum(proportions)*len(idx_k)).astype(int)[:-1]
                idx_batch = [idx_j + idx.tolist() for idx_j,idx in zip(idx_batch,np.split(idx_k,proportions))]
                min_size = min([len(idx_j) for idx_j in idx_batch])
            try_cnt += 1

        for j in range(self.cfg.num_clients):
            dataidx_map[j] = idx_batch[j]

        X, y, statistic = self.assign(dataset_content, dataset_label, dataidx_map, X, y, statistic)

        return X, y, statistic

In [ ]:
class ExtPartitioner(BasePartitioner):
    # https://arxiv.org/abs/2311.03154
    def __init__(self, cfg):
        super().__init__(cfg)
    
    def parition(self, dataset_content, dataset_label, least_samples, dataidx_map):
        X = [[] for _ in range(self.cfg.num_clients)]
        y = [[] for _ in range(self.cfg.num_clients)]
        statistic = [[] for _ in range(self.cfg.num_clients)]

        C = self.cfg.data.class_per_client
        
        '''The first level: allocate labels to clients
        clientidx_map (dict, {label: clientidxclass_per_client}), e.g., C=2, cfg.num_clients=5, cfg.data.num_classes=10
            {0: [0, 1], 1: [1, 2], 2: [2, 3], 3: [3, 4], 4: [4, 5], 5: [5, 6], 6: [6, 7], 7: [7, 8], 8: [8, 9], 9: [9, 0]}
        '''
        min_size_per_label = 0
        # You can adjust the `min_require_size_per_label` to meet you requirements
        min_require_size_per_label = max(C * self.cfg.num_clients // self.cfg.data.num_classes // 2, 1)
        if min_require_size_per_label < 1:
            raise ValueError
        clientidx_map = {}
        while min_size_per_label < min_require_size_per_label:
            # initialize
            for k in range(self.cfg.data.num_classes):
                clientidx_map[k] = []
            # allocate
            for i in range(self.cfg.num_clients):
                labelidx = np.random.choice(range(self.cfg.data.num_classes), C, replace=False)
                for k in labelidx:
                    clientidx_map[k].append(i)
            min_size_per_label = min([len(clientidx_map[k]) for k in range(self.cfg.data.num_classes)])
        
        '''The second level: allocate data idx'''
        dataidx_map = {}
        y_train = dataset_label
        min_size = 0
        min_require_size = 10
        K = self.cfg.data.num_classes
        N = len(y_train)
        print("\n*****clientidx_map*****")
        print(clientidx_map)
        print("\n*****Number of clients per label*****")
        print([len(clientidx_map[i]) for i in range(len(clientidx_map))])

        # ensure per client' sampling size >= min_require_size (is set to 10 originally in [3])
        while min_size < min_require_size:
            idx_batch = [[] for _ in range(self.cfg.num_clients)]
            # for each class in the dataset
            for k in range(K):
                idx_k = np.where(y_train == k)[0]
                np.random.shuffle(idx_k)
                proportions = np.random.dirichlet(np.repeat(self.cfg.data.alpha, self.cfg.num_clients))
                # cfg.data.balance
                # Case 1 (original case in Dir): cfg.data.balance the number of sample per client
                proportions = np.array([p * (len(idx_j) < N / self.cfg.num_clients and j in clientidx_map[k]) for j, (p, idx_j) in enumerate(zip(proportions, idx_batch))])
                # Case 2: Don't cfg.data.balance
                #proportions = np.array([p * (j in label_netidx_map[k]) for j, (p, idx_j) in enumerate(zip(proportions, idx_batch))])
                proportions = proportions / proportions.sum()
                proportions = (np.cumsum(proportions) * len(idx_k)).astype(int)[:-1]
                # process the remainder samples
                '''Note: Process the remainder data samples (yipeng, 2023-11-14).
                There are some cases that the samples of class k are not allocated completely, i.e., proportions[-1] < len(idx_k)
                In these cases, the remainder data samples are assigned to the last client in `clientidx_map[k]`.
                '''
                if proportions[-1] != len(idx_k):
                    for w in range(clientidx_map[k][-1], self.cfg.num_clients-1):
                        proportions[w] = len(idx_k)
                idx_batch = [idx_j + idx.tolist() for idx_j, idx in zip(idx_batch, np.split(idx_k, proportions))] 
                min_size = min([len(idx_j) for idx_j in idx_batch])

        for j in range(self.cfg.num_clients):
            np.random.shuffle(idx_batch[j])
            dataidx_map[j] = idx_batch[j]

        X, y, statistic = self.assign(dataset_content, dataset_label, dataidx_map, X, y, statistic)

        return X, y, statistic

In [ ]:
path = '/home/ahmed/Ahmed-home/1- Projects/Research/publications/2024/letter 1/code/PFLlib/dataset/Cifar10/'

In [ ]:
def read_data(path, idx, is_train=True):
    if is_train:
        train_data_dir = os.path.join(path, 'train')

        train_file = os.path.join(train_data_dir, str(idx) + '.npz')
        with open(train_file, 'rb') as f:
            train_data = np.load(f, allow_pickle=True)['data'].tolist()

        return train_data

    else:
        test_data_dir = os.path.join(path, 'test/')

        test_file = test_data_dir + str(idx) + '.npz'
        with open(test_file, 'rb') as f:
            test_data = np.load(f, allow_pickle=True)['data'].tolist()
    
        return test_data

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()